对比open3d中各种降采样方法的效果



In [19]:
import open3d as o3d
__all__ = [o3d]
print(o3d.__version__)

0.5.0.0


加载深度图并转换为点云

In [20]:
import matplotlib.pyplot as plot
import numpy as np
depth_image_path = "/media/yumi/Datas/6D_Dataset/BOP_Dataste/" \
                   "LM-O/train_pbr/000000/depth/000000.png"
depth_raw = o3d.read_image(depth_image_path)
image_width = 640
image_height = 480
fx = 572.4114
fy = 573.57043
cx = 325.2611
cy = 242.04899
intrinsic = o3d.PinholeCameraIntrinsic(
    image_width, image_height, fx, fy, cx, cy)
extrinsic = np.identity(4)
print(extrinsic)
depth_scale = 10
depth_trunc = 3000.0 # 超过depth_trunc的深度值将会被设为0

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [21]:
np_depth = np.array(depth_raw)
depth_raw = o3d.geometry.Image(np_depth)
print(depth_raw)
print(type(depth_raw))
print(type(intrinsic))



Image of size 640x480, with 1 channels.
Use numpy.asarray to access buffer data.
<class 'open3d.open3d.geometry.Image'>
<class 'open3d.open3d.camera.PinholeCameraIntrinsic'>


In [22]:
color = True
pcd = o3d.create_point_cloud_from_depth_image(
    depth=depth_raw, intrinsic=intrinsic, extrinsic=extrinsic,
    depth_scale=depth_scale, depth_trunc=depth_trunc, stride=int(1))
if not color:
    pcd.paint_uniform_color([0, 255, 0])
o3d.visualization.draw_geometries([pcd])



voxel down sample 并返回降采样之后的点云索引
索引是一个n*8的矩阵,每一行里面几个点云的索引

In [23]:
print(type(pcd))
max_bound = pcd.get_max_bound()
min_bound = pcd.get_min_bound()
voxel_size = 10
pcd_dow, point_idx = o3d.voxel_down_sample_and_trace(
    pcd, voxel_size=voxel_size,
    min_bound=min_bound, max_bound=max_bound)
if not color:
    pcd.paint_uniform_color([0, 255, 0])
o3d.visualization.draw_geometries([pcd_dow])
# o3d.write_point_cloud("/home/yumi/Desktop/SampleDate/pcd_down0731.ply", pcd_dow)


<class 'open3d.open3d.geometry.PointCloud'>


每一行中点云的索引和降采样后点云的关系:
降采样后的点和索引对应的点并不相同


In [25]:
# 通过索引提取点云并输出坐标
np_pcd_dow = np.asarray(pcd_dow.points)  # 将点云转换成numpy矩阵
pcd_dow_point0 = pcd_dow.points[0]
pcd_point291122 = pcd.points[1]
print(point_idx[0])
print(pcd_dow_point0)
print(pcd_point291122)

[    -1     -1     -1     -1     -1 227544     -1     -1]
[362.18337498 295.08861458 714.29998779]
[-1372.81388539 -1022.68434363  2423.39990234]


In [26]:
row_idx = 54
point_idx4 =point_idx[row_idx]
point_set_4 = []
for i , idx in enumerate(point_idx4):
    if idx > 0 :
        point = pcd.points[idx]
        point_set_4.append(point)

point_4 = pcd_dow.points[row_idx]
distance = []
# for i , point in enumerate(point_set_4):
#     point = np.asarray(point)
#     point_4 = np.asarray(point_4)
#     distance1 = np.square((point_4[0]-point[0])) + \
#                 np.square((point_4[1]-point[1])) + \
#                 np.square((point_4[2]-point[2]))
#     distance.append(distance1)
#
# print(distance)
point_4_dis = point_set_4 - point_4
print(point_4_dis)
distance = np.square(point_4_dis)
print(distance)
point_distance = []
for i , dis in enumerate(distance):
    point_distance.append(dis.sum())
print(point_distance)

[[-0.45336796  0.37352117 -3.60000992]
 [ 3.56300993  1.4873903  -3.30002213]
 [ 0.15123893 -0.01559465  0.3000145 ]
 [ 4.2110676   1.14498089  0.69997787]]
[[2.05542504e-01 1.39518061e-01 1.29600714e+01]
 [1.26950398e+01 2.21232990e+00 1.08901460e+01]
 [2.28732149e-02 2.43193068e-04 9.00086977e-02]
 [1.77330904e+01 1.31098124e+00 4.89969025e-01]]
[13.305131976707122, 25.79751569155538, 0.11312510573211378, 19.53404062637031]


在全图中显示降采样的点和索引对应的点

In [27]:
mesh_sphere = o3d.create_mesh_sphere(radius=2.0)
mesh_sphere.compute_vertex_normals()
mesh_frame = o3d.create_mesh_coordinate_frame(size=100, origin=[0,0,0])
mesh_sphere.paint_uniform_color([0,255,0])
H = np.identity(4)

for i , cord in enumerate(point_4):
    H[i][3] = cord
# print(H)
mesh_sphere.transform(H)

o3d.visualization.draw_geometries([mesh_sphere, mesh_frame])



In [28]:
def creat_sphere_set(central_point, other_points,
                    color = None ):
    if color is None:
	    color = [[2550.5, 0, 0], [0, 255, 0]]
    mesh_sphere_all = []
    mesh_sphere_central = o3d.create_mesh_sphere(radius=1.0)
    mesh_sphere_central.paint_uniform_color(color[0])
    mesh_sphere_central.compute_vertex_normals()
    H_central = np.identity(4)
    for i , cord in enumerate(central_point):
        H_central[i][3] = cord
    mesh_sphere_central.transform(H_central)
    mesh_sphere_all.append(mesh_sphere_central)


    for i, point in enumerate(other_points):
        mesh_sphere_other = o3d.create_mesh_sphere(radius=2.0)
        mesh_sphere_other.paint_uniform_color(color[1])
        mesh_sphere_other.compute_vertex_normals()
        H_other = np.identity(4)
        for i, cord in enumerate(point):
            H_other[i][3] = cord
        mesh_sphere_other.transform(H_other)
        mesh_sphere_all.append(mesh_sphere_other)

    return mesh_sphere_all

mesh_sphere_all = creat_sphere_set(
    point_4, point_set_4
)
# 将中心点与其他点用线连起来
point_set =[point_4.tolist()]
for i, point in enumerate(point_set_4):
    point_set.append(point.tolist())

line_set = []
for i in range(len(point_set)):
    if i != 0:
        line_set.append([0,i])

lines = o3d.LineSet()
lines.points = o3d.Vector3dVector(point_set)
lines.lines = o3d.Vector2iVector(line_set)
#lines.paint_uniform_color([0,0,255])

central_frame = o3d.create_mesh_coordinate_frame(size=100, origin=point_4)


show_list = mesh_sphere_all
show_list.append(lines)
#pcd.paint_uniform_color([255,255,255])
#show_list.append(pcd)
#show_list.append(central_frame)
o3d.visualization.draw_geometries(show_list)




选择离网格中心最近的点作为采样点

In [29]:
idx_mask = point_idx > 0
idx_true = idx_mask.nonzero()
row_list = idx_true[0].tolist()
col_list = idx_true[1].tolist()
sampled_points_idx = []
sampled_points_num = point_idx.shape[0]
# 循环次数太多,速度过慢
for i in range(sampled_points_num):
    row_list_idx = row_list.index(i)
    col_idx = col_list[row_list_idx]
    sampled_points_idx.append(point_idx[i][col_idx])

voxel_random_downsample = o3d.select_down_sample(pcd, sampled_points_idx)
o3d.visualization.draw_geometries([voxel_random_downsample])

In [31]:
# 选择每一行中,数值最大的索引
max_idx_in_row = np.max(point_idx, axis=1)
voxel_max_idx_sample = o3d.select_down_sample(pcd, max_idx_in_row)
o3d.visualization.draw_geometries([voxel_max_idx_sample])




如果采样后的点数大于指定值,通过KNN选择指定点数的点云

In [32]:
print(len(sampled_points_idx))
point_num = len(sampled_points_idx)
max_point_num = 20000

# 索引的大小和距离无关
if point_num > max_point_num:
    remove_num = max_point_num - point_num
    retain_idx = sampled_points_idx[:remove_num]
    remove_idx = sampled_points_idx[max_point_num:]
    # np.array(voxel_random_downsample.colors)[remove_idx[1:], :] = [0, 0, 1]
    # np.array(voxel_random_downsample.colors)[retain_idx[1:], :] = [0, 1, 0]
    retain_points = o3d.select_down_sample(pcd, retain_idx)
    remove_points = o3d.select_down_sample(pcd, remove_idx)
    o3d.visualization.draw_geometries([retain_points])
    o3d.visualization.draw_geometries([remove_points])


43161


In [33]:

if point_num > max_point_num:
    pcd_dow_tree = o3d.KDTreeFlann(voxel_random_downsample)
    k, idx , _ = pcd_dow_tree.search_knn_vector_3d([0,0,0], 20000)
    retain_points = o3d.select_down_sample(voxel_random_downsample, idx)
    remove_points = o3d.select_down_sample(voxel_random_downsample, idx, True)
    o3d.visualization.draw_geometries([retain_points])
    o3d.visualization.draw_geometries([remove_points])



恢复原图的索引

In [ ]:
original_idx = sampled_points_idx
print("original idx len:{}".format(len(original_idx)))
idx = np.asarray(idx).tolist()
original_idx = np.asarray(original_idx)
after_KNN = original_idx[idx]
print("after KNN len :{}".format(len(after_KNN)))
after_KNN_points = o3d.select_down_sample(pcd, after_KNN)
o3d.visualization.draw_geometries([after_KNN_points])




将原图的索引放到color中,降采样后再取出

In [36]:
# 保留1.5米以内的点
pcd_tree = o3d.geometry.KDTreeFlann(pcd)
k, pcd_radius_1500_idx , pcd_radius_1500_idx_what = pcd_tree.search_radius_vector_3d([0,0,0], 1500)
pcd_1500 = o3d.select_down_sample(pcd, pcd_radius_1500_idx)
o3d.visualization.draw_geometries([pcd_1500])

In [ ]:
pcd_1500_idx = np.asarray(pcd_radius_1500_idx)
pcd_1500_idx = np.array([pcd_1500_idx, pcd_1500_idx, pcd_1500_idx]).transpose()
color_idx = pcd_1500_idx.tolist()
pcd_1500.colors = o3d.Vector3dVector(color_idx)
idx_from_color = np.asarray(pcd_1500.colors)
pcd_1500_tree = o3d.geometry.KDTreeFlann(pcd_1500)
k, pcd_KNN_100_idx, _ = pcd_1500_tree.search_knn_vector_3d([0,0,0], 100)
pcd_KNN_100 = o3d.select_down_sample(pcd_1500, pcd_KNN_100_idx)
idx_original = np.asarray(pcd_KNN_100.colors).astype(np.int)
idx_original = idx_original[:,0].tolist()
